In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import sys
import dateutil.parser

from __future__ import print_function

In [2]:
sys.path.insert(0, os.path.abspath('..'))

from UKMovementSensing import dataprep

## Filepaths

In [56]:
rootpath = '/media/sf_VBox_Shared/London/raw/next161/'
annotations_path = rootpath + 'tud_next161_deb.csv'
wearcodes_path = rootpath + 'wearcodes.csv'

In [57]:
filepath = rootpath + 'accelerometer_5second/'

## Daylight saving time

## Load and preprocess annotations

In [108]:
annotations = dataprep.process_annotations(annotations_path)
print(annotations.shape)
annotations.head()

Number of missing start and end times: 0 0
Differens between start and end not always 10 minutes! in 322 cases. First 10:
                    start_time                  end_time
119  2015-02-15 23:50:00+00:00 2015-02-15 00:00:00+00:00
263  2015-02-20 23:50:00+00:00 2015-02-20 00:00:00+00:00
407  2015-03-08 23:50:00+00:00 2015-03-08 00:00:00+00:00
551  2015-03-10 23:50:00+00:00 2015-03-10 00:00:00+00:00
695  2015-03-01 23:50:00+00:00 2015-03-01 00:00:00+00:00
839  2015-03-05 23:50:00+00:00 2015-03-05 00:00:00+00:00
983  2015-03-01 23:50:00+00:00 2015-03-01 00:00:00+00:00
1127 2015-03-02 23:50:00+00:00 2015-03-02 00:00:00+00:00
1271 2015-03-21 23:50:00+00:00 2015-03-21 00:00:00+00:00
1415 2015-03-24 23:50:00+00:00 2015-03-24 00:00:00+00:00
Multiple end times per slot. first 10:
Slot
1     [04:10:00, 03:10:00]
2     [04:20:00, 03:20:00]
3     [04:30:00, 03:30:00]
4     [04:40:00, 03:40:00]
5     [04:50:00, 03:50:00]
6     [05:00:00, 04:00:00]
7     [05:10:00, 04:10:00]
8     [05:20:00, 0

,serflag,tud_day,Slot,start_time,end_time,act,act_label
0,101390011_100,1,1,2015-02-15 04:00:00+00:00,2015-02-15 04:10:00+00:00,1.0,Sleeping and resting (including sick in bed)
1,101390011_100,1,2,2015-02-15 04:10:00+00:00,2015-02-15 04:20:00+00:00,1.0,Sleeping and resting (including sick in bed)
2,101390011_100,1,3,2015-02-15 04:20:00+00:00,2015-02-15 04:30:00+00:00,1.0,Sleeping and resting (including sick in bed)
3,101390011_100,1,4,2015-02-15 04:30:00+00:00,2015-02-15 04:40:00+00:00,1.0,Sleeping and resting (including sick in bed)
4,101390011_100,1,5,2015-02-15 04:40:00+00:00,2015-02-15 04:50:00+00:00,1.0,Sleeping and resting (including sick in bed)


In [96]:
# To check: Do we have any gaps?
for i in range(1, annotations.shape[0]):
    if annotations['serflag'][i]==annotations['serflag'][i-1] and annotations['tud_day'][i]==annotations['tud_day'][i-1]:
        if (annotations['end_time'][i-1] != annotations['start_time'][i]):
            print(annotations.loc[[i-1, i],['start_time', 'end_time']])
            print(annotations['end_time'][i-1] - annotations['start_time'][i])

                    start_time                  end_time
1703 2015-03-29 22:50:00+00:00 2015-03-29 23:00:00+00:00
1704 2015-03-30 00:00:00+00:00 2015-03-30 00:10:00+00:00
-1 days +23:00:00
                    start_time                  end_time
2423 2015-03-29 22:50:00+00:00 2015-03-29 23:00:00+00:00
2424 2015-03-30 00:00:00+00:00 2015-03-30 00:10:00+00:00
-1 days +23:00:00
                     start_time                  end_time
14375 2015-03-29 22:50:00+00:00 2015-03-29 23:00:00+00:00
14376 2015-03-30 00:00:00+00:00 2015-03-30 00:10:00+00:00
-1 days +23:00:00
                     start_time                  end_time
21431 2015-03-29 22:50:00+00:00 2015-03-29 23:00:00+00:00
21432 2015-03-30 00:00:00+00:00 2015-03-30 00:10:00+00:00
-1 days +23:00:00
                     start_time                  end_time
25463 2015-03-29 22:50:00+00:00 2015-03-29 23:00:00+00:00
25464 2015-03-30 00:00:00+00:00 2015-03-30 00:10:00+00:00
-1 days +23:00:00
                     start_time               

## Join with wearcodes

In [110]:
wearcodes = dataprep.read_wearcodes(wearcodes_path)
print(wearcodes.shape)
wearcodes.head()

(161, 8)


,Monitor,Day1,Day2,binFile,serflag,fileSize,day1Ready,day2Ready
0,18643,2015-03-25,2015-03-28,__018643_2015-03-31 15-31-31.bin,41350051_100,477086967,True,True
1,19959,2015-03-10,2015-03-14,__019959_2015-03-20 12-13-46.bin,41410011_100,649365449,True,True
2,20443,2015-03-28,2015-03-30,__020443_2015-04-07 11-00-31.bin,23040141_100,434219870,True,True
3,20447,2015-03-19,2015-03-22,__020447_2015-03-31 09-45-39.bin,41880051_100,791590930,True,True
4,20515,2015-02-21,2015-02-27,__020515_2015-03-10 13-56-58.bin,42370091_100,791590016,True,True


In [116]:
annotations_codes = dataprep.join_wearcodes(wearcodes_path, annotations)
print(annotations_codes.shape)
annotations_codes.head()

(46368, 14)


,serflag,tud_day,Slot,start_time,end_time,act,act_label,Monitor,Day1,Day2,binFile,fileSize,day1Ready,day2Ready
0,101390011_100,1,1,2015-02-15 04:00:00+00:00,2015-02-15 04:10:00+00:00,1.0,Sleeping and resting (including sick in bed),21563,2015-02-15,2015-02-20,__021563_2015-03-06 09-56-08.bin,791586540,True,True
1,101390011_100,1,2,2015-02-15 04:10:00+00:00,2015-02-15 04:20:00+00:00,1.0,Sleeping and resting (including sick in bed),21563,2015-02-15,2015-02-20,__021563_2015-03-06 09-56-08.bin,791586540,True,True
2,101390011_100,1,3,2015-02-15 04:20:00+00:00,2015-02-15 04:30:00+00:00,1.0,Sleeping and resting (including sick in bed),21563,2015-02-15,2015-02-20,__021563_2015-03-06 09-56-08.bin,791586540,True,True
3,101390011_100,1,4,2015-02-15 04:30:00+00:00,2015-02-15 04:40:00+00:00,1.0,Sleeping and resting (including sick in bed),21563,2015-02-15,2015-02-20,__021563_2015-03-06 09-56-08.bin,791586540,True,True
4,101390011_100,1,5,2015-02-15 04:40:00+00:00,2015-02-15 04:50:00+00:00,1.0,Sleeping and resting (including sick in bed),21563,2015-02-15,2015-02-20,__021563_2015-03-06 09-56-08.bin,791586540,True,True


## Load and preprocess data

In [168]:
dfs = dataprep.process_data(annotations_codes, filepath)

starttime of data does not correspond with starttime of annotations!
2015-03-25 04:00:00 2015-03-28 04:00:00+00:00
File not found:  __019959_2015-03-20 12-13-46.bin_day1.csv
File not found:  __019959_2015-03-20 12-13-46.bin_day2.csv
starttime of data does not correspond with starttime of annotations!
2015-03-28 04:00:00 2015-03-30 03:00:00+00:00
starttime of data does not correspond with starttime of annotations!
2015-03-19 04:00:00 2015-03-22 04:00:00+00:00
starttime of data does not correspond with starttime of annotations!
2015-02-21 04:00:00 2015-02-27 04:00:00+00:00
starttime of data does not correspond with starttime of annotations!
2015-03-06 04:00:00 2015-03-08 04:00:00+00:00
starttime of data does not correspond with starttime of annotations!
2015-03-14 04:00:00 2015-03-16 04:00:00+00:00
starttime of data does not correspond with starttime of annotations!
2015-02-15 04:00:00 2015-02-18 04:00:00+00:00
File not found:  __020572_2015-03-13 14-15-25.bin_day2.csv
File not found:  _

In [159]:
example_key = dfs.keys()[0]
binfile, day = example_key
df = dfs[example_key]
annotations_group = byName.get_group((binfile, day))
df.head()

,acceleration,anglex,angley,anglez,invalid,filename,Slot
timestamp,,,,,,,
2015-02-26 04:00:00,0.007361,6.54791335069374,-3.09760534819941,82.750551647458,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:05,0.006762,6.54812920492664,-3.06154453017752,82.7654299778353,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:10,0.007433,6.54904658541647,-2.90828605358448,82.8286628819388,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:15,0.006907,6.54907369988162,-3.09815240202131,82.7492657005061,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:20,0.007692,6.54791335069374,-3.09760534819941,82.750551647458,1.0,__021380_2015-03-06 11-07-21.bin_day1,1


In [157]:
firsttime = df.index[0]
print(firsttime.tz_localize('UTC') == min(annotations_group.start_time))
df['Slot'] = df.index - firsttime
df['Slot'] = [int(np.floor(s.total_seconds()/600))+1 for s in df['Slot']]
df.head()

True


,acceleration,anglex,angley,anglez,invalid,filename,Slot
timestamp,,,,,,,
2015-02-26 04:00:00,0.007361,6.54791335069374,-3.09760534819941,82.750551647458,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:05,0.006762,6.54812920492664,-3.06154453017752,82.7654299778353,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:10,0.007433,6.54904658541647,-2.90828605358448,82.8286628819388,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:15,0.006907,6.54907369988162,-3.09815240202131,82.7492657005061,1.0,__021380_2015-03-06 11-07-21.bin_day1,1
2015-02-26 04:00:20,0.007692,6.54791335069374,-3.09760534819941,82.750551647458,1.0,__021380_2015-03-06 11-07-21.bin_day1,1


In [38]:
#Check if the date in the accelerometer data correpsonds with the date in wearcodes
#wearcodes.groupby('binFile')
for binFile, day in dfs.keys():
    df = dfs[(binFile, day)]
    wearcode = wearcodes[wearcodes['binFile']==binFile]
    realday = wearcode.to_dict()['Day'+str(day)].values()[0].date()
    file_date_start = df.index[0]
    file_date_end = df.index[-1]
    if file_date_start.date() !=  realday:
        print(binFile+'_day'+str(day), "\t", str(realday), "\t", file_date_start, file_date_end,"\t")

__023456_2015-03-06 09-27-16.bin_day2 	 2015-02-21 	 2015-02-20 04:00:00 2015-02-21 02:59:55 	
__022783_2015-03-18 14-22-46.bin_day2 	 2015-03-08 	 2015-03-05 04:00:00 2015-03-06 02:59:55 	
__022631_2015-03-30 14-50-34.bin_day2 	 2015-03-17 	 2015-03-14 04:00:00 2015-03-15 03:59:55 	
__022058_2015-03-31 09-46-00.bin_day2 	 2015-03-23 	 2015-03-22 04:00:00 2015-03-23 02:59:55 	
__021179_2015-03-30 10-32-48.bin_day2 	 2015-03-21 	 2015-03-16 04:00:00 2015-03-17 02:59:55 	
__021808_2015-03-19 14-10-59.bin_day2 	 2015-03-08 	 2015-03-03 04:00:00 2015-03-04 02:59:55 	
__020780_2015-03-30 14-15-03.bin_day2 	 2015-03-12 	 2015-03-08 04:00:00 2015-03-09 02:59:55 	
__021671_2015-04-08 11-03-31.bin_day2 	 2015-03-21 	 2015-03-17 04:00:00 2015-03-18 02:59:55 	
__022799_2015-04-07 13-43-53.bin_day2 	 2015-03-23 	 2015-03-22 04:00:00 2015-03-23 03:59:55 	
__021563_2015-03-06 09-56-08.bin_day2 	 2015-02-20 	 2015-02-15 04:00:00 2015-02-16 03:59:55 	
__022868_2015-03-11 14-06-20.bin_day2 	 2015-03-05

In [34]:
print(wearcode.to_dict()['Day1'].values()[0].date())
df.index

2015-02-22


DatetimeIndex(['2015-02-22 04:00:00', '2015-02-22 04:00:05',
               '2015-02-22 04:00:10', '2015-02-22 04:00:15',
               '2015-02-22 04:00:20', '2015-02-22 04:00:25',
               '2015-02-22 04:00:30', '2015-02-22 04:00:35',
               '2015-02-22 04:00:40', '2015-02-22 04:00:45',
               ...
                               'NaT',                 'NaT',
                               'NaT',                 'NaT',
                               'NaT',                 'NaT',
                               'NaT',                 'NaT',
                               'NaT',                 'NaT'],
              dtype='datetime64[ns]', name=u'timestamp', length=17280, freq=None)

In [ ]:
for binFile, day in dfs.keys():
    df = dfs[(binFile, day)]
    

In [35]:
?pd.read_csv

In [ ]:
merged_directory = os.path.join(rootpath, 'merged_5sec')
if not os.path.exists(merged_directory):
    os.makedirs(merged_directory)

In [ ]:
for binfile, day in dfs.keys():
    df = dfs[(binFile, day)]
    df.to_csv(os.path.join(merged_directory, binfile+'_day'+day+'.csv'))